In [1]:
import os, sys
from pathlib import Path

src_path = Path(os.getcwd()).resolve().parents[1]  
sys.path.append(str(src_path))

In [2]:
import os
print(os.getcwd())

/Users/seobi/PythonProjects/ProjectML/src/agents/fairy


In [3]:
from agents.fairy.fairy_state import FairyState, FairyOutput
FairyState()

{}

In [4]:
from langchain.chat_models import init_chat_model
from enums.LLM import LLM

llm = init_chat_model(model=LLM.GPT4_1_MINI,temperature=0.4)
# parser_llm = llm.with_structured_output(FairyOutput)

# result = parser_llm.invoke("안녕하세요")
# result



/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from langgraph.graph import START, END
from prompts.promptmanager import PromptManager
from prompts.prompt_type.fairy.FairyPromptType import FairyPromptType
from langchain_core.messages import SystemMessage, HumanMessage
from agents.fairy.temp_string import temp_heroine_info

system_dungeon_prompt = PromptManager(FairyPromptType.FAIRY_DUNGEON_SYSTEM).get_prompt(
    heroine_status = """
    공격력 10
    방어력 5
    HP 40/100
    MP 20/50
    """,
    heroine_info = temp_heroine_info,
    dungeon_info = "총 4층으로 이루어진 던전 방의 갯수는 층당 각각 5~7개 사이",
    event_info = "",
    floor = "2",
    level = "난이도 하",
    current_room = "슬라임 3마리, 울프 5마리"
)
result = llm.invoke([SystemMessage(content = system_dungeon_prompt)] + [HumanMessage(content = "슬라임 어떻게 공격해?")])

In [6]:

result.content

'슬라임은 공격력이 약하고 방어력이 낮아서 기본 공격으로도 쉽게 상대할 수 있어요. 다만, 슬라임은 분열하는 성질이 있으니 한 번에 여러 마리를 공격하는 스킬이나 범위 공격이 있다면 효과적이에요. 레티아, 쌍검으로 빠르게 연속 공격해서 한 마리씩 확실히 처리하는 게 안전할 거예요! 너무 오래 끌면 슬라임이 늘어나서 골치 아파질 수 있으니까요~'

In [7]:
system_guild_prompt = PromptManager(FairyPromptType.FAIRY_GUILD_SYSTEM).get_prompt(
    heroine_info = temp_heroine_info,
    heroine_status = """
    공격력 10
    방어력 5
    HP 40/100
    MP 20/50
    """
)

In [8]:
result = llm.invoke([SystemMessage(content = system_guild_prompt)] + [HumanMessage(content = "나 심심해")])

In [9]:
print(result.content)

어머, 심심하다니! 그럼 내가 재미있는 이야기 하나 들려줄까? 아니면 레티아랑 같이 던전 탐험하면서 숨겨진 비밀을 찾아볼까? 아니면 그냥 가벼운 수다나 할래? 뭐든 말만 해봐, 내가 곁에서 팍팍 도와줄게! 😊


# 몬스터 공략 (주로 보스몬스터 위주로)
 1. 현재방의 몬스터 상세 정보 필요 -> 상세정보 Search RAG (K = 1)
 2. 몬스터 찾기 멀티쿼리 필요 -> 사용자는 몬스터 명을 모를 확률이 큼 => 멀티쿼리로 현재방중 몬스터 특징에 따라 몬스터 찾기
 3. 몬스터 공략법 RAG 필요 -> 찾은 몬스터의 공략 해결책을 찾는 서칭 필요 

# 이벤트 공략 
 - 캐시 DB로부터 생성된 이벤트 정보 GET -> 힌트 위주로 제공 

# 던전 안내
 - 캐시 DB로 부터 생성된 던전의 전체 특징을 요약 후 브리핑 

# 인터렉션 
 - 방 불키기, 물약 사용, 아이템 사용 return 액션, 사용ID(Option)

# 멀티턴 전략 
 - 이전 대화 요약본 저장 

